# Numpy and Matplotlib

What are they?
- [NumPy](https://numpy.org): number manipulation & crunching
- [Matplotlib](https://matplotlib.org): visualisations and plots

Why do we learn NumPy? Its syntax has inspired the whole field of ML (TensorFlow, PyTorch, JAX, TensorFlowJS and therefore ml5js). Note: it is also similar to [Matlab](https://en.wikipedia.org/wiki/MATLAB).

### Installation
On google collab most of the content in this notebook will not need installation. If using your local Python installation install with conda by writing the following in the terminal:
```
conda install -c conda-forge numpy
conda install -c conda-forge matplotlib
```

---

# Numpy

Numpy is a *numerical computation* package allowing us to do all sort of things with `array`s.

### Vocab note

- a **matrix** is a 2D array
- a **tensor** is everything above that (3D, 4D arrays, etc.) (and because it's so common now, people talk about tensors for all sizes of arrays...)

In [ ]:
import numpy as np # import the package and rename it 'np', the convention

In [ ]:
x = np.array([0, 1, 2, 3]) # think of this as a Python list on steroids
print(x)
print(len(x)) # like a Python list
print(x[0:2]) # slicing

## Array creation

### arange

In [ ]:
x = np.arange(9) # like Python range
print(x)

#### zeros, ones

In [ ]:
x = np.zeros((5, 10)) # 5 rows, 10 columns, try also `np.ones`
print(x)
print(len(x)) # ! number of *rows*, like a list of lists
print(x.size) # total number of elements

In [ ]:
print(x.shape) # `shape`: this is the information we need
rows, cols = x.shape
# rows = x.shape[0]  # the same
# cols = x.shape[1]

#### eye

In [ ]:
np.eye(3) # a 3x3 matrix with ones in the diagonal

#### linspace

In [ ]:
np.linspace(-1, 1, 9) # 9 numbers equally spaced between -1 and 1

## Indexing and slicing



In [ ]:
x = np.arange(9).reshape(3,3)
x

In [ ]:
x[2] # the third row

In [ ]:
x[2, 0] # a single element!
        # same as x[0][2] for a list of lists in plain Python

In [ ]:
x[1, :] # first row, all elements (`:` means "everything")
        # can you display only the second column?

## Reshaping & transposing arrays

In ML we constantly reshape data tensors to suit our needs!

In [ ]:
x = np.arange(2*3*4)
print(x.shape)
print(x) # in NumPy, the shape of a 1D array is a tuple of one el (note the comma: (24,)!)

In [ ]:
print(x.reshape((2,3,4))) # the shape must divide the total number of els without remainder!
                          # "two matrices with three rows of four columns"

In [ ]:
print(x.reshape((2,-1))) # '-1': calculate that dimension for me

In [ ]:
x_reshaped = x.reshape((2, 2, -1))
print(x_reshaped)
print(x_reshaped.shape) # see how '-1' calculated '6' for the last dim

In [ ]:
print(x.reshape((3,-1))) # '-1': calculate that dimension for me

We can also *add dimensions*:

In [ ]:
print(x)

print(np.expand_dims(x, 0))   # wrap the array with one more dimension
# print(x[np.newaxis, ...])   # the same. '...' means "fill in the remaining dimensions"
# print(x[None, ...])         # the same

In [ ]:
column_x = x[..., None]
print(column_x) # adding a dimension at the end turns the row vector into a column one
print(column_x.shape)

... and also *remove them* (`np.squeeze` by default removes all dimensions of size one).

In [ ]:
x_squeezed = np.squeeze(column_x)
print(x_squeezed)
print(x_squeezed.shape) # back where we started

### Transposing

AKA "rows become columns and columns become rows":

In [ ]:
x = np.arange(9).reshape((3,3))
print(x)
print()
print(x.T)               # transpose the matrix (= flip it along the diagonal)
# print(np.transpose(x)) # the same

Algebra is not everyone's cup of tea, but is a fundamental part of ML!
For more:
- the fantastic [Essence of Linear Algebra by 3Blue1Brown](https://www.youtube.com/watch?v=fNk_zzaMoSs&list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab)
- [a matrix algebra primer](https://www.statlect.com/matrix-algebra/).

## More fun: images are arrays!

To practice slicing it is useful to actually look at an image, which we will represent as a numpy array.

We will use the [Pillow](https://realpython.com/image-processing-with-the-python-pillow-library/) library which can be accessed with the `PIL` module. If the code below does not work install Pillow with:

```bash
conda install -c conda-forge pillow
```

Note that we are not limited to 2d arrays, for example a R,G,B image could be
represented as a `(rows, columns, 3)` array where the last 3 dimensions each
represents a channel of the image... more on this later when we look at
plotting.

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# In Colab
# https://stackoverflow.com/a/23489503
import requests
from io import BytesIO
url = "https://raw.githubusercontent.com/jchwenger/DMLAP/main/python/images/spock.jpg"
response = requests.get(url)
img = np.array(Image.open(BytesIO(response.content)))

# Locally
# img = np.array(Image.open('images/spock.jpg')) # also possible: plt.imread(). see below
print(img.shape)                               # our image is an array of shape (w,h,channels)

In [ ]:
plt.imshow(img)

In [ ]:
plt.imshow(img[170:220, 100:480]) # slicing in the first two dims!

Now let's look at a single channel. We need to specify the display in greyscale. There are [many other options](https://matplotlib.org/stable/tutorials/colors/colormaps.html#sequential).

In [ ]:
plt.imshow(img[:, :, 0], cmap='gray') # try viridis, plasma, inferno...

In [ ]:
plt.imshow(img[:, ::-1]) # flipped left to right: how do you flip it upside down?

In [ ]:
plt.imshow(img[:, :, ::-1]) # reversing the channels!

### Note: handling batches

Almost systematically in ML, we deal with *batches* of images, rather than single ones, to take advantage of parallel computing. That can be handled by adding an additional (batch) dimension at the front of our image array/tensor:

In [ ]:
print(img.shape)
img_batch = img.reshape([1, 688, 500, 3])  # note: these numbers must be exact otherwise np complains!
# img_batch = img.reshape([1, *img.shape]) # the same, programmatically unpacking the shape array
# img_batch = np.expand_dims(img, 0)       # the same: 'expands' the zeroth dimension
# img_batch = img[np.newaxis, ...]         # the same newaxis adds an axis, '...' means "the rest of the shape"
# img_batch = img[None, ...]               # the same! None in this context works like np.newaxis
print(img_batch.shape)

Other times, you just want to flatten everything:

In [ ]:
print(img.shape)
print(img.ravel()[None, ...].shape) # `ravel` flattens, then we add a batch dimension
print(img.reshape([1, -1]).shape)   # the same, '-1' means 'calculate the size of this dimension for me

## Stacking & combining arrays

### concatenate

In [ ]:
a = np.arange(3)
b = np.arange(3,6)
a,b

In [ ]:
np.concatenate([a,b])

### tile

In [ ]:
np.tile(a, 3) # repeat the array three times

### vstack, hstack (concatenating vertically, horizontally)

Along an *existing* axis, like concatenate (discussion [here](https://stackoverflow.com/a/61760947)).

In [ ]:
a = np.arange(2)
b = np.arange(2,6).reshape(2,2)

a, b

In [ ]:
np.vstack([a, b]) # vertically

In [ ]:
a_col = a[...,None] # to stack horizontally, we turn a into a column vector
print(a_col)
np.hstack([a_col, b]) 

### stack

Along a *new* axis (discussion [here](https://stackoverflow.com/a/61760947)).

In [ ]:
a = np.arange(3)
b = np.arange(3,6)

a,b

In [ ]:
np.stack([a,b], axis=0) # creates a new container 'in axis 0' and puts the two arrays in it

In [ ]:
np.stack([a,b], axis=1) # # creates new containers 'in axis 1' and puts the contents of the arrays in them

These functions will only work if the size of the arrays we want to combine is **compatible**, i.e. we can only stack horizontally if arrays have the same number of rows, and we can only stack vertically if arrays have the number of columns.

More:
- [joining arrays](https://numpy.org/doc/stable/reference/routines.array-manipulation.html#joining-arrays)
- [splitting arrays](https://numpy.org/doc/stable/reference/routines.array-manipulation.html#splitting-arrays)
- [tiling arrays](https://numpy.org/doc/stable/reference/routines.array-manipulation.html#tiling-arrays)
- [adding and removing elements](https://numpy.org/doc/stable/reference/routines.array-manipulation.html#adding-and-removing-elements)

## Math operations and broadcasting

- [matrix multiplication](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html#numpy-matmul)
- [dot product](https://numpy.org/doc/stable/reference/generated/numpy.dot.html)
- see [here](https://stackoverflow.com/a/34142617)

In [ ]:
a = np.arange(4).reshape((2,2))
print(a)
print(a @ a)            # matrix multiplication
# print(np.matmul(a,a)) # same
# print(a.dot(a))       # dot product, in this case the same

#### broadcasting

This is the real superpower of NumPy (copied by all ML libraries).

Often you wish to apply the same operations to all elements of a tensor. In plain Python, you need a loop. In NumPy, you use **broadcasting** (which much, *much* faster).

In [ ]:
a = np.ones((3,3))
a

In [ ]:
print(a * 9) # works with all basic operations: + - * / **

In [ ]:
a = np.ones((3,3))
b = np.arange(3)
a,b

In [ ]:
a.shape, b.shape # b is transformed to shape (1,3), then copied three times

In [ ]:
a * b # broadcasting vertically

In [ ]:
a.shape, b[...,None].shape 

In [ ]:
a * b[...,None] # broadcasting horizontally

#### Beware, there are rules!

![Sasha Rush broadcasting tweet](https://raw.githubusercontent.com/jchwenger/AI/main/2-building-blocks/images/tf/srush-broadcasting.png)

See also [this tutorial](https://numpy.org/doc/stable/user/basics.broadcasting.html).

## Random numbers


In [ ]:
print(np.random.uniform())                    # a number between 0 and 1
print(np.random.uniform(-100, 100))           # a number between -100 and 100
print(np.random.uniform(-10, 10, size=(2,3))) # an array of shape 2x3 with numbers between -10 and 10

In [ ]:
np.random.normal() # a number from a Gaussian distribution: mean 0, standard deviation of 1

In [ ]:
np.random.randint(0, 10) # an integer between 0 and 10

### Note: random seeds

If we want to always get the same random values, we can set a number as a **seed** to the random number generator.

In [ ]:
np.random.seed(100)
print(np.random.uniform(-1, 1, 3))
np.random.seed(100) # try and comment this out
print(np.random.uniform(-1, 1, 3))

### Note: uniform vs normal?

- **Uniform**: flat distribution ('pure' random)
- **Normal**: Gaussian, Bell curve, Normal distribution, centred around around a *mean*, with tails on both ends determined by the *standard deviation*

(Don't worry about the plotting code, it's a [histogram](https://matplotlib.org/stable/gallery/statistics/hist.html#sphx-glr-gallery-statistics-hist-py).)

In [ ]:
import matplotlib.pyplot as plt

uniform_throws = np.random.uniform(size=(1000,))
normal_throws = np.random.normal(size=(1000,))
                                   
fig, (ax1, ax2) = plt.subplots(1,2, sharey=True, tight_layout=True) # 1,2 means: 1 row & 2 columns
fig.suptitle("Throwing one 'uniform' and one 'normal' die 1000 times, plotting the results")
ax1.set_title("Uniform")                                            # ax1 is our 'axes' (the plot object of subplot 1)
ax1.hist(uniform_throws, bins=25)
ax2.set_title("Normal/Gaussian")                                    # ax2 is our 'axes' (the plot object of subplot 2)
ax2.hist(normal_throws, bins=25)
plt.show()

## More operations on arrays

### min, max

In [ ]:
z = np.random.uniform(-10, 10, 10) # 10 random numbers between -10 and 10
print(z)
print(np.max(z)) # the biggest number, try np.min()

In [ ]:
(z - np.min(z)) / (np.max(z) - np.min(z)) # useful to normalize the array

### argmin, argmax

Often instead of the *value* (min or max), you need its **index**!

In [ ]:
p = [0.3, 0.5, 0.1, 0.1]
print(p)
print(np.argmax(p)) # *index* of the max, try np.argmax

### sum

We can specify *along which axis* to perform the operation.

(See also [`np.average` and others](https://numpy.org/doc/stable/reference/routines.statistics.html#averages-and-variances).)

In [ ]:
z = np.ones((3, 4))
print(z)
print()

print(np.sum(z))             # grand total
print(np.sum(z, axis=0)) # sum along columns
print(np.sum(z, axis=1))     # sum along rows

Often we want to keep the dimensions intact:

In [ ]:
np.sum(z, keepdims=True) # keepdims exists in many NumPy functions

### Note: duplicate methods

A lot of these methods also exist as methods, so you can write `z.min()`, `z.sum()`, etc.

More resources:
- [NumPy: the absolute basics for beginners](https://numpy.org/doc/stable/user/absolute_beginners.html)
- [NumPy quickstart](https://numpy.org/doc/stable/user/quickstart.html)
- [NumPy fundamentals](https://numpy.org/doc/stable/user/basics.html)
- [Even more learning resources](https://numpy.org/learn/)

---

# Beyond NumPy: TensorFlow

For a notebook covering many similar concepts in TensorFlow:
- see [`1.first-steps-tensorflow.ipynb`](https://github.com/jchwenger/AI/blob/main/labs/1-lab/1.first-steps-tensorflow.ipynb)
- and its empty, 'quiz' form [`1.first-steps-tensorflow.QUIZ.ipynb`](https://github.com/jchwenger/AI/blob/main/labs/1-lab/1.first-steps-tensorflow.QUIZ.ipynb)).

---

# Matplotlib (plotting)

One of the most widely used tools for plotting tasks in Python is a package called [Matplotlib](https://matplotlib.org).

In [ ]:
import matplotlib.pyplot as plt    # import pyplot submodule, rename it 'plt'

In [ ]:
x = np.linspace(0, np.pi * 4, 100) # 100 numbers between 0 and 4 * pi
y = np.sin(x)                      # the sinus of each of these (broadcasting!)

plt.figure(figsize=(10, 5))        # create figure, specifying its size as a tuple (optional, try without it)
plt.plot(x, y)                     # plot the numbers
plt.show()                         # show the plot

### Multiple pieces of data in one plot

In [ ]:
plt.figure(figsize=(10, 5))            # create the figure
plt.title("Cosine and Sine")           # title

plt.plot(x, np.cos(x), label='Cosine') # some data
plt.plot(x, np.sin(x), label='Sine')   # more data

plt.legend()                           # without this, the 'labels' aren't shown
# plt.savefig('myplot.pdf')            # you can save to pdf, png, etc.
plt.show()                             # show the figure

See [the `plot` documentation](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html), and [the examples](https://matplotlib.org/stable/gallery/index.html).

### Multiple plots, aka subplots

In [ ]:
plt.figure(figsize=(10, 5))

ax1 = plt.subplot(1, 2, 1)   # (1,2,1) means: 1 row & 2 columns, setting up subplot 1
ax1.set(title="Cosine")      # ax1 is our 'axes' (the plot object of subplot 1)
ax1.plot(x, np.cos(x))

ax2 = plt.subplot(1, 2, 2)   # (1,2,2) means: 1 row & 2 columns, setting up subplot 2
ax2.set(title="Sine")        # ax2 is our 'axes' (the plot object of subplot 2)
ax2.plot(x, np.sin(x))

plt.show()

## Loading and showing images

In [ ]:
# In Colab, see requests code above
img = plt.imread(BytesIO(response.content), format="jpeg") # format necessary: https://stackoverflow.com/a/64100066

# Locally
# img = plt.imread('images/spock.jpg') # pyplot can load images directly
plt.figure()
plt.imshow(img)
plt.axis('off') # comment this to get the ticks/numbers back
plt.show()

In [ ]:
print(img.shape) # this is a np.array
print(img.dtype) # type 'uint8

This is a subtlety that we will have do deal with especially when working with images.

We can still do slicing etc for example this will show a detail of only one channel of the image:



In [ ]:
plt.figure()
plt.imshow(img[180:220, 220:280, 2], interpolation='nearest')
plt.show()

Note that we use an optional parameter `interpolation='nearest'`. This indicates that we do not want to filter the image when scaling it and it results in the pixels being shown as distinct squares. More [here](https://matplotlib.org/stable/gallery/images_contours_and_fields/interpolation_methods.html) and [here](https://matplotlib.org/stable/gallery/images_contours_and_fields/image_antialiasing.html).



More resources:
- [Pyplot tutorial](https://matplotlib.org/stable/tutorials/introductory/pyplot.html)
- [Image tutorial](https://matplotlib.org/stable/tutorials/introductory/images.html)
- [Quick start guide](https://matplotlib.org/stable/tutorials/introductory/quick_start.html)